# DL Lab 5

Implement Transfer learning for different CNN models

Reference link: https://towardsdatascience.com/a-comprehensive-hands-on-guide-to-transfer-learning-with-real-world-applications-in-deep-learning-212bf3b2f27a

In [1]:
import tensorflow as tf

import cv2

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

## Transfer Learning on CIFAR10

In [2]:
(x_t, y_t), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

x_train, x_val, y_train, y_val = train_test_split(x_t, y_t)

# Normalizing pixel values
x_train, x_val, x_test = x_train / 255, x_val / 255, x_test / 255

In [3]:
x_train[0].shape

(32, 32, 3)

## Model

We will try to freeze different parts of the model to see which of the pre-trained weights still stay relevant.

In [4]:
def setupBase(pix=72, ip=32, resize=True):
    
    inputs = tf.keras.Input(shape=(ip, ip, 3))
    upscale = tf.keras.layers.Lambda(lambda x: 
                                     tf.image.resize(
                                             images = x,
                                             size = (pix, pix),
                                             method=tf.image.ResizeMethod.BILINEAR))(inputs)
    if resize:
        return tf.keras.applications.Xception(
                                        weights='imagenet',
                                        input_tensor=upscale,
                                        input_shape=(pix, pix, 3),
                                        include_top=False,
                                        pooling='max'
                                        )
    else:
        return tf.keras.applications.Xception(
                                        weights='imagenet',
                                        input_shape=(pix, pix, 3),
                                        include_top=False,
                                        pooling='max'
                                        )

def makeModel(base):
    
    x = base.output
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(64, activation='relu') (x)
    x = tf.keras.layers.Dropout(0.4) (x)
    x = tf.keras.layers.Dense(10, activation='softmax') (x)
    model = tf.keras.Model(inputs=base.input, outputs=x)

    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
    optimizer_fn = tf.keras.optimizers.Adam(learning_rate=0.001)

    model.compile(optimizer=optimizer_fn,
                  loss=loss_fn,
                  metrics=['accuracy'])

    return model

def trainModel(model):
    history = model.fit(
                    x_train,
                    y_train,
                    validation_data=(x_val, y_val),
                    epochs=5,
                    batch_size=128,
                    verbose=1,
                    workers=12,
                    use_multiprocessing=True)
    return history

### Freezing 50-100%

In [5]:
# Create Base model
base = setupBase()

# Freezing
for i in base.layers[len(base.layers)//2:len(base.layers)]:
    i.trainable = False
    
model = makeModel(base)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 72, 72, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 35, 35, 32)   864         lambda[0][0]                     
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 35, 35, 32)   128         block1_conv1[0][0]               
______________________________________________________________________________________________

In [6]:
history = trainModel(model)

Epoch 1/5
293/293 [==============================] - 127s 410ms/step - loss: 0.6412 - accuracy: 0.8009 - val_loss: 0.4240 - val_accuracy: 0.8658
Epoch 2/5
293/293 [==============================] - 174s 596ms/step - loss: 0.2624 - accuracy: 0.9183 - val_loss: 0.4020 - val_accuracy: 0.8742
Epoch 3/5
293/293 [==============================] - 193s 657ms/step - loss: 0.1751 - accuracy: 0.9441 - val_loss: 0.4184 - val_accuracy: 0.8802
Epoch 4/5
293/293 [==============================] - 193s 658ms/step - loss: 0.1368 - accuracy: 0.9569 - val_loss: 0.4193 - val_accuracy: 0.8939
Epoch 5/5
293/293 [==============================] - 192s 657ms/step - loss: 0.1014 - accuracy: 0.9682 - val_loss: 0.4087 - val_accuracy: 0.8940


In [7]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 17s 52ms/step - loss: 0.4202 - accuracy: 0.8970


[0.42024925351142883, 0.8970000147819519]

### Freezing layers 25-75%

In [8]:
# Create Base model
base = setupBase()

# Freezing
for i in base.layers[len(base.layers)//4:3*len(base.layers)//4]:
    i.trainable = False
    
model = makeModel(base)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 72, 72, 3)    0           input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 35, 35, 32)   864         lambda_1[0][0]                   
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 35, 35, 32)   128         block1_conv1[0][0]               
____________________________________________________________________________________________

In [9]:
history = trainModel(model)

Epoch 1/5
293/293 [==============================] - 199s 663ms/step - loss: 0.6159 - accuracy: 0.8145 - val_loss: 0.6568 - val_accuracy: 0.8311
Epoch 2/5
293/293 [==============================] - 194s 661ms/step - loss: 0.3370 - accuracy: 0.8969 - val_loss: 0.4508 - val_accuracy: 0.8682
Epoch 3/5
293/293 [==============================] - 195s 665ms/step - loss: 0.3148 - accuracy: 0.9088 - val_loss: 0.5229 - val_accuracy: 0.8455
Epoch 4/5
293/293 [==============================] - 193s 660ms/step - loss: 0.2043 - accuracy: 0.9351 - val_loss: 0.4037 - val_accuracy: 0.8825
Epoch 5/5
293/293 [==============================] - 193s 659ms/step - loss: 0.1154 - accuracy: 0.9636 - val_loss: 0.3247 - val_accuracy: 0.9100


In [10]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 15s 49ms/step - loss: 0.3435 - accuracy: 0.9048


[0.34350836277008057, 0.9047999978065491]

### Freezing layers 0-50%

In [11]:
# Create Base model
base = setupBase()

# Freezing
for i in base.layers[:len(base.layers)//2]:
    i.trainable = False
    
model = makeModel(base)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 72, 72, 3)    0           input_3[0][0]                    
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 35, 35, 32)   864         lambda_2[0][0]                   
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 35, 35, 32)   128         block1_conv1[0][0]               
____________________________________________________________________________________________

In [12]:
history = trainModel(model)

Epoch 1/5
293/293 [==============================] - 130s 433ms/step - loss: 0.7791 - accuracy: 0.7581 - val_loss: 0.7708 - val_accuracy: 0.8053
Epoch 2/5
293/293 [==============================] - 126s 431ms/step - loss: 0.4323 - accuracy: 0.8663 - val_loss: 0.5562 - val_accuracy: 0.8318
Epoch 3/5
293/293 [==============================] - 126s 431ms/step - loss: 0.2887 - accuracy: 0.9131 - val_loss: 0.5093 - val_accuracy: 0.8530
Epoch 4/5
293/293 [==============================] - 126s 431ms/step - loss: 0.2560 - accuracy: 0.9212 - val_loss: 0.4371 - val_accuracy: 0.8660
Epoch 5/5
293/293 [==============================] - 126s 431ms/step - loss: 0.1657 - accuracy: 0.9500 - val_loss: 0.4873 - val_accuracy: 0.8748


In [13]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 15s 48ms/step - loss: 0.5361 - accuracy: 0.8621


[0.5361389517784119, 0.8621000051498413]

## Transfer learning on another dataset

### About dataset

Fine-Grained Visual Classification of Aircraft (FGVC-Aircraft) is a benchmark dataset for the fine grained visual categorization of aircraft. The dataset contains 10,200 images of aircraft, with 100 images for each of 102 different aircraft model variants, most of which are airplanes. The (main) aircraft in each image is annotated with a tight bounding box and a hierarchical airplane model label.

Aircraft models are organized in a four-levels hierarchy. The four levels, from finer to coarser, are:

Model, e.g. Boeing 737-76J. Since certain models are nearly visually indistinguishable, this level is not used in the evaluation.
Variant, e.g. Boeing 737-700. A variant collapses all the models that are visually indistinguishable into one class. The dataset comprises 102 different variants.
Family, e.g. Boeing 737. The dataset comprises 70 different families.
Manufacturer, e.g. Boeing. The dataset comprises 29 different manufacturers.
The data is divided into three equally-sized training, validation and test subsets. The first two sets can be used for development, and the latter should be used for final evaluation only.

Link- https://www.robots.ox.ac.uk/~vgg/data/fgvc-aircraft/

In this experiment, we are only going to try and predict the manufacturer and nothing else. Since it is a 29 class classification, making this will be a challlenge all in itself.

### Preprocessing

In [14]:
imsize = 128

In [15]:
path = "./fgvc-aircraft-2013b/data/"

In [16]:
train_ids = open(path+'images_manufacturer_train.txt', 'r')
val_ids = open(path+'images_manufacturer_val.txt', 'r')
test_ids = open(path+'images_manufacturer_test.txt', 'r')

In [17]:
trIds = train_ids.read().split('\n')
vaIds = val_ids.read().split('\n')
teIds = test_ids.read().split('\n')
trIds[:5]

['1025794 Boeing',
 '1340192 Boeing',
 '0056978 Boeing',
 '0698580 Boeing',
 '0450014 Boeing']

In [18]:
x_train, x_val = [], []

In [19]:
y_train, y_val = [], []

In [20]:
trid, tvid, teid = [], [], []
for i in trIds:
    val = i.split()
    if len(val) > 0:
        ids = val[0]
        man = val[1]
        trid.append(ids)
        y_train.append(man)
        
for i in vaIds:
    val = i.split()
    if len(val) > 0:
        ids = val[0]
        man = val[1]
        tvid.append(ids)
        y_val.append(man)

In [21]:
image = None
impath = ''
for i in range(len(trid)):
    impath = path+'images/'+trid[i]+'.jpg'
    image = cv2.resize(cv2.imread(impath), (imsize, imsize), interpolation=cv2.INTER_AREA)
    x_train.append(image)
    if image is None:
        print(i)

In [22]:
for i in range(len(tvid)):
    impath = path+'images/'+tvid[i]+'.jpg'
    image = cv2.resize(cv2.imread(impath), (imsize, imsize), interpolation=cv2.INTER_AREA)
    x_val.append(image)
    if image is None:
        print(i)

In [23]:
x_train = np.array(x_train, dtype='uint8') / 255.0

In [24]:
x_val = np.array(x_val, dtype='uint8') / 255.0

In [25]:
y_train = np.array(pd.get_dummies(y_train))
y_val = np.array(pd.get_dummies(y_val))

In [26]:
y_train.shape, y_val.shape

((3334, 29), (3333, 29))

### Model Building

In [27]:
base = setupBase(pix=imsize, ip=imsize, resize=False)

# Freezing
for i in base.layers[:len(base.layers)//2]:
    i.trainable = False

In [28]:
x = base.output
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(64, activation='relu') (x)
x = tf.keras.layers.Dropout(0.4) (x)
x = tf.keras.layers.Dense(y_train.shape[1], activation='softmax') (x)
model = tf.keras.Model(inputs=base.input, outputs=x)

loss_fn = tf.keras.losses.CategoricalCrossentropy()
optimizer_fn = tf.keras.optimizers.Adam()

model.compile(optimizer=optimizer_fn,
              loss=loss_fn,
              metrics=['accuracy'])

model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 63, 63, 32)   864         input_5[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 63, 63, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 63, 63, 32)   0           block1_conv1_bn[0][0]            
____________________________________________________________________________________________

In [29]:
history = model.fit(x_train, y_train,
                    validation_data=(x_val, y_val),
                    epochs=20,
                    batch_size=16,
                    verbose=2,
                    workers=10,
                    use_multiprocessing=True)

Epoch 1/20
209/209 - 29s - loss: 3.1913 - accuracy: 0.1854 - val_loss: 2.7763 - val_accuracy: 0.2295
Epoch 2/20
209/209 - 23s - loss: 2.6394 - accuracy: 0.2493 - val_loss: 3.7081 - val_accuracy: 0.1830
Epoch 3/20
209/209 - 23s - loss: 2.4196 - accuracy: 0.2789 - val_loss: 2.2413 - val_accuracy: 0.2766
Epoch 4/20
209/209 - 24s - loss: 2.2268 - accuracy: 0.3164 - val_loss: 3.5145 - val_accuracy: 0.2946
Epoch 5/20
209/209 - 26s - loss: 2.0040 - accuracy: 0.3548 - val_loss: 1.8671 - val_accuracy: 0.3906
Epoch 6/20
209/209 - 26s - loss: 1.7571 - accuracy: 0.4169 - val_loss: 1.7739 - val_accuracy: 0.4503
Epoch 7/20
209/209 - 26s - loss: 1.7617 - accuracy: 0.4328 - val_loss: 4.7351 - val_accuracy: 0.3234
Epoch 8/20
209/209 - 26s - loss: 1.4845 - accuracy: 0.5012 - val_loss: 2.3067 - val_accuracy: 0.4197
Epoch 9/20
209/209 - 26s - loss: 1.2997 - accuracy: 0.5606 - val_loss: 1.6543 - val_accuracy: 0.4752
Epoch 10/20
209/209 - 26s - loss: 1.0770 - accuracy: 0.6479 - val_loss: 1.7165 - val_accura

### Model Testing

In [30]:
x_test, y_test = [], []

In [31]:
for i in teIds:
    val = i.split()
    if len(val) > 0:
        ids = val[0]
        man = val[1]
        teid.append(ids)
        y_test.append(man)

In [32]:
for i in range(len(teid)):
    impath = path+'images/'+teid[i]+'.jpg'
    image = cv2.resize(cv2.imread(impath), (imsize, imsize), interpolation=cv2.INTER_AREA)
    x_test.append(image)
    if image is None:
        print(i)

In [33]:
x_test = np.array(x_test, dtype='uint8') / 255.0

In [34]:
y_test = np.array(pd.get_dummies(y_test))

In [35]:
y_test.shape

(3333, 29)

In [36]:
model.evaluate(x_test, y_test)

105/105 [==============================] - 8s 66ms/step - loss: 2.1062 - accuracy: 0.5983


[2.1061646938323975, 0.5982598066329956]